In [1]:
import pandas as pd
import glob
import os
from sklearn.preprocessing import MinMaxScaler

#### 1. 캘린더 생성

In [2]:
data_path = '데이터'

visitor_files_pattern = os.path.join(data_path, '제주내국인_입도통계_*.csv')
visitor_files_pattern

'데이터\\제주내국인_입도통계_*.csv'

In [3]:
file_rentcar = os.path.join(data_path, '렌터카 가동률_2023_202508.csv')
file_rentcar

'데이터\\렌터카 가동률_2023_202508.csv'

In [4]:
file_charter_bus = os.path.join(data_path, '전세버스 가동률_2023_202508.csv')
file_charter_bus

'데이터\\전세버스 가동률_2023_202508.csv'

In [5]:
print(f"입도객 파일 패턴: {visitor_files_pattern}")
print(f"렌터카 파일 경로: {file_rentcar}")
print(f"전세버스 파일 경로: {file_charter_bus}")

입도객 파일 패턴: 데이터\제주내국인_입도통계_*.csv
렌터카 파일 경로: 데이터\렌터카 가동률_2023_202508.csv
전세버스 파일 경로: 데이터\전세버스 가동률_2023_202508.csv


데이터 로딩

In [6]:
visitor_files = glob.glob(visitor_files_pattern)

df_list = []
for file in visitor_files:
    df_list.append(pd.read_csv(file))

df_visitors_raw = pd.concat(df_list, ignore_index=True)

# '구분' == '형태' 필터링 (이중 집계 방지)
df_visitors_filtered = df_visitors_raw[df_visitors_raw['구분'] == '형태'].copy()

# '년월' 기준 '총 입도객' 합산
df_total_visitors = df_visitors_filtered.groupby('년월')['입도객 수'].sum().reset_index()
df_total_visitors = df_total_visitors.rename(columns={'입도객 수': '총 입도객'})

# '년', '월' 컬럼 생성 (Merge 용)
df_total_visitors['년'] = df_total_visitors['년월'] // 100
df_total_visitors['월'] = df_total_visitors['년월'] % 100

print("--- 입도객 데이터 처리 완료 (데이터 하위 5개) ---")
print(df_total_visitors.tail())

--- 입도객 데이터 처리 완료 (데이터 하위 5개) ---
        년월    총 입도객     년  월
75  202504  1143910  2025  4
76  202505  1252408  2025  5
77  202506  1200051  2025  6
78  202507  1312159  2025  7
79  202508  1329111  2025  8


In [7]:
df_rentcar_raw = pd.read_csv(file_rentcar, encoding='UTF-8-SIG')
df_rentcar = df_rentcar_raw[['년', '월', '가동률']].rename(columns={'가동률': '렌터카 가동률'})
df_rentcar.head()

,년,월,렌터카 가동률
0,2025.0,1.0,63.5
1,2025.0,2.0,60.8
2,2025.0,3.0,60.1
3,2025.0,4.0,68.3
4,2025.0,5.0,71.0


In [8]:
df_charter_bus_raw = pd.read_csv(file_charter_bus, encoding='utf-8-sig')
df_charter_bus = df_charter_bus_raw[['년', '월', '가동률']].rename(columns={'가동률': '전세버스 가동률'})
df_charter_bus.head()

,년,월,전세버스 가동률
0,2025.0,1.0,9.0
1,2025.0,2.0,9.4
2,2025.0,3.0,22.7
3,2025.0,4.0,39.6
4,2025.0,5.0,39.2


데이터 병합

In [9]:
df_merged = pd.merge(df_total_visitors, df_rentcar, on=['년', '월'], how='inner')
df_merged = pd.merge(df_merged, df_charter_bus, on=['년', '월'], how='inner')
df_merged.head()

,년월,총 입도객,년,월,렌터카 가동률,전세버스 가동률
0,202301,1032565,2023,1,71.1,12.2
1,202302,1054299,2023,2,72.7,13.9
2,202303,1069614,2023,3,68.8,30.8
3,202304,1157268,2023,4,71.7,38.8
4,202305,1180708,2023,5,68.3,43.0


정규화 `'MinMaxScailer'`활용

In [10]:
cols_to_normalize = ['총 입도객', '렌터카 가동률', '전세버스 가동률']
scaler = MinMaxScaler()

In [11]:
normalized_cols = [f'{col}_norm' for col in cols_to_normalize]
df_merged[normalized_cols] = scaler.fit_transform(df_merged[cols_to_normalize])

In [12]:
df_merged['관광 포화 지수'] = df_merged[normalized_cols].mean(axis=1)

df_merged['날짜'] = pd.to_datetime(df_merged['년'].astype(str) + '-' + df_merged['월'].astype(str) + '-01')

In [13]:
df_merged.head()

,년월,총 입도객,년,월,렌터카 가동률,전세버스 가동률,총 입도객_norm,렌터카 가동률_norm,전세버스 가동률_norm,관광 포화 지수,날짜
0,202301,1032565,2023,1,71.1,12.2,0.407511,0.564103,0.084433,0.352015,2023-01-01
1,202302,1054299,2023,2,72.7,13.9,0.450935,0.646154,0.129288,0.408792,2023-02-01
2,202303,1069614,2023,3,68.8,30.8,0.481534,0.446154,0.575198,0.500962,2023-03-01
3,202304,1157268,2023,4,71.7,38.8,0.656664,0.594872,0.786280,0.679272,2023-04-01
4,202305,1180708,2023,5,68.3,43.0,0.703496,0.420513,0.897098,0.673702,2023-05-01


In [14]:
# colunms 순서 정리
final_columns = [
    '날짜', '년', '월', '년월',
    '총 입도객', '렌터카 가동률', '전세버스 가동률', # 원본값
    '총 입도객_norm', '렌터카 가동률_norm', '전세버스 가동률_norm', # 정규화된 값
    '관광 포화 지수'
]

In [15]:
df_final = df_merged[final_columns].sort_values(by='날짜')
df_final.head()

,날짜,년,월,년월,총 입도객,렌터카 가동률,전세버스 가동률,총 입도객_norm,렌터카 가동률_norm,전세버스 가동률_norm,관광 포화 지수
0,2023-01-01,2023,1,202301,1032565,71.1,12.2,0.407511,0.564103,0.084433,0.352015
1,2023-02-01,2023,2,202302,1054299,72.7,13.9,0.450935,0.646154,0.129288,0.408792
2,2023-03-01,2023,3,202303,1069614,68.8,30.8,0.481534,0.446154,0.575198,0.500962
3,2023-04-01,2023,4,202304,1157268,71.7,38.8,0.656664,0.594872,0.786280,0.679272
4,2023-05-01,2023,5,202305,1180708,68.3,43.0,0.703496,0.420513,0.897098,0.673702


In [16]:
output_file = os.path.join(data_path, 'golden_compass_base_index.csv')
df_final.to_csv(output_file, index=False, encoding='utf-8-sig')